In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("../datasets/fashion-mnist/fashion-mnist_train.csv")
train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train_data.shape

(60000, 785)

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [6]:
train_data.isna().any()

label       False
pixel1      False
pixel2      False
pixel3      False
pixel4      False
pixel5      False
pixel6      False
pixel7      False
pixel8      False
pixel9      False
pixel10     False
pixel11     False
pixel12     False
pixel13     False
pixel14     False
pixel15     False
pixel16     False
pixel17     False
pixel18     False
pixel19     False
pixel20     False
pixel21     False
pixel22     False
pixel23     False
pixel24     False
pixel25     False
pixel26     False
pixel27     False
pixel28     False
pixel29     False
            ...  
pixel755    False
pixel756    False
pixel757    False
pixel758    False
pixel759    False
pixel760    False
pixel761    False
pixel762    False
pixel763    False
pixel764    False
pixel765    False
pixel766    False
pixel767    False
pixel768    False
pixel769    False
pixel770    False
pixel771    False
pixel772    False
pixel773    False
pixel774    False
pixel775    False
pixel776    False
pixel777    False
pixel778    False
pixel779  

In [5]:
colnames = train_data.columns.values.tolist()
X = train_data[colnames[1:]]
y = train_data[colnames[0]] # Label

In [6]:
X.shape, y.shape

((60000, 784), (60000,))

In [7]:
# Create a StandarScaler instance and transform the data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
scaler = StandardScaler()

X = scaler.fit_transform(X)
X.shape

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


(60000, 784)

In [8]:
# Training and validation 
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [9]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((42000, 784), (42000,), (18000, 784), (18000,))

In [11]:
# Process X_train data with SMOTE
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X_train, y_train)
type(X_res) # No longer pandas dataframe, but we will convert tp H20 dataframe

numpy.ndarray

In [12]:
X_res.shape

(42400, 784)

In [13]:
np.sum(y_res)/len(y_res)

4.5

In [16]:
# H20 requires composite data with both predictors and target
y_res = y_res.reshape(len(y_res), 1)
y_res.shape

(42400, 1)

In [17]:
X = np.hstack((X_res, y_res))
X.shape

(42400, 785)

In [24]:
# start h2o
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_191"; Java(TM) SE Runtime Environment (build 1.8.0_191-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/5t/92c414c509j78wb5j48yqqcr0000gn/T/tmpy9k14yn7
  JVM stdout: /var/folders/5t/92c414c509j78wb5j48yqqcr0000gn/T/tmpy9k14yn7/h2o_diegogarcia_viana_started_from_python.out
  JVM stderr: /var/folders/5t/92c414c509j78wb5j48yqqcr0000gn/T/tmpy9k14yn7/h2o_diegogarcia_viana_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.2
H2O cluster version age:,2 months and 15 days
H2O cluster name:,H2O_from_python_diegogarcia_viana_sz3lgo
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [28]:
# Transform data to h2o dataframe
df = h2o.H2OFrame(X)
len(df.columns)

Parse progress: |█████████████████████████████████████████████████████████| 100%


785

In [29]:
df.shape

(42400, 785)

In [30]:
df.columns

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26',
 'C27',
 'C28',
 'C29',
 'C30',
 'C31',
 'C32',
 'C33',
 'C34',
 'C35',
 'C36',
 'C37',
 'C38',
 'C39',
 'C40',
 'C41',
 'C42',
 'C43',
 'C44',
 'C45',
 'C46',
 'C47',
 'C48',
 'C49',
 'C50',
 'C51',
 'C52',
 'C53',
 'C54',
 'C55',
 'C56',
 'C57',
 'C58',
 'C59',
 'C60',
 'C61',
 'C62',
 'C63',
 'C64',
 'C65',
 'C66',
 'C67',
 'C68',
 'C69',
 'C70',
 'C71',
 'C72',
 'C73',
 'C74',
 'C75',
 'C76',
 'C77',
 'C78',
 'C79',
 'C80',
 'C81',
 'C82',
 'C83',
 'C84',
 'C85',
 'C86',
 'C87',
 'C88',
 'C89',
 'C90',
 'C91',
 'C92',
 'C93',
 'C94',
 'C95',
 'C96',
 'C97',
 'C98',
 'C99',
 'C100',
 'C101',
 'C102',
 'C103',
 'C104',
 'C105',
 'C106',
 'C107',
 'C108',
 'C109',
 'C110',
 'C111',
 'C112',
 'C113',
 'C114',
 'C115',
 'C116',
 'C117',
 'C118',
 'C119',
 'C120',
 'C121',
 'C122',
 'C123',
 

In [33]:
# Predictors and target columns
X_columns = df.columns[:784] # only column names, no data
y_columns = df.columns[784]
y_columns

'C785'

In [34]:
# For classification, target column must be a factor (h2o requirement)
df[y_columns] = df[y_columns].asfactor()

In [36]:
#Build a deeplearning model on balanced data
#     http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/deep-learning.html
dl_model = H2ODeepLearningEstimator(epochs=500,
                                    distribution = 'bernoulli',                 # Response has two levels
                                    missing_values_handling = "MeanImputation", # Not needed by us
                                    variable_importances=True,
                                    nfolds = 2,                           # CV folds
                                    fold_assignment = "Stratified",       # Each fold must be sampled carefully
                                    keep_cross_validation_predictions = True,  # For analysis
                                    balance_classes=False,                # SMOTE is not provided by h2o
                                    standardize = True,                   # z-score standardization
                                    activation = 'RectifierWithDropout',  # Default dropout is 0.5
                                    hidden = [32,32],                  # ## more hidden layers -> more complex interactions
                                    stopping_metric = 'logloss',
                                    loss = 'CrossEntropy')

In [40]:
# Train out model
from time import time
start = time()
dl_model.train(X_columns,
               y_columns,
               training_frame=df)
end = time()
print("Time: {} minutes".format((end-start)/60))

deeplearning Model Build progress: |██████████████████████████████████████| 100%
Time: 5.1089846849441525 minutes


In [42]:
# Make predictions
y_val = (y_val.values).reshape(len(y_val), 1)
y_val.shape

(18000, 1)

In [45]:
X_val = np.hstack((X_val, y_val))
X_val.shape

(18000, 785)

In [49]:
X_val = h2o.H2OFrame(X_val)
X_val["C785"] = X_val["C785"].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [50]:
# Make prediction on X_val
pred = dl_model.predict(X_val[:, 0:784])
pred.shape

deeplearning prediction progress: |███████████████████████████████████████| 100%


(18000, 11)

In [51]:
pred.as_data_frame().head()

,predict,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,9,0.000014,9.173111e-05,0.000008,0.000100,0.000006,1.381942e-03,0.000013,5.498747e-02,3.115775e-04,9.430865e-01
1,0,0.771701,6.820298e-08,0.001280,0.005205,0.000029,2.027199e-11,0.221784,5.007423e-11,6.467627e-07,1.303611e-12
2,7,0.000211,5.430477e-06,0.000316,0.000187,0.000046,4.260212e-03,0.000772,9.939978e-01,5.224325e-05,1.519017e-04
3,8,0.072395,5.421305e-04,0.037247,0.017181,0.032852,1.160177e-02,0.110916,1.152289e-02,7.000277e-01,5.714771e-03
4,8,0.059858,7.478956e-05,0.059534,0.014073,0.043340,4.935676e-03,0.105972,3.000624e-04,7.115576e-01,3.554686e-04


In [54]:
# Back to pandas dataframe
xe = X_val["C785"].as_data_frame()
xe["result"] = pred[0].as_data_frame()
xe.head()

,C785,result
0,9,9
1,0,0
2,7,7
3,8,8
4,8,8


In [64]:
out = (xe["result"] == xe["C785"])
print("Accurary: {:.4f}".format((np.sum(out)/len(xe))*100))

Accurary: 83.0778
